In [1]:
import os

import numpy as np
import pickle
from tqdm import tqdm

from ArcParams_class import ArcParams
import utils

import cpp_soft_info as csi
from soft_info import inv_qubit_mapping
from Scratch import postselect_calib_data



def process_pSoft(pSoft, estim0Mat, estim1Mat, threshold):
    pSoft_copy = pSoft.copy()
    mask = ((estim0Mat < threshold) & (estim1Mat < threshold))
    pSoft_copy[mask] = 0.5-1e-8

    # calculate the filtered ratio
    ratio = np.sum(mask) / (pSoft.shape[0]*pSoft.shape[1])
    return pSoft_copy, ratio



files = [f for f in os.listdir('data/IQ_data_X/') if os.path.isfile(os.path.join('data/IQ_data_X/', f))]

for file in tqdm(files):

    # Load and correct the data
    with open('data/IQ_data_X/' + file, 'rb') as f:
        data = pickle.load(f)
    with open('data/params/' + file, 'rb') as f:
        params = pickle.load(f)


    cor_data = []
    for idx, data_list in enumerate(data):
        vstack = np.vstack(data_list)
        cor_data.append(vstack)

    # Create all memories
    # NEW DATA FORMAT: |0> MM, |1> MM, |0> MXM, |1> MXM, memory
    all_memories_dbl = {qubit: {} for qubit in range(127)}
    all_memories_dbl_X = {qubit: {} for qubit in range(127)}
    for qubit in all_memories_dbl.keys():
        all_memories_dbl[qubit]['mmr_0'] = cor_data[0][:, qubit]
        all_memories_dbl[qubit]['mmr_1'] = cor_data[1][:, qubit]
        all_memories_dbl[qubit]['mmr_0_scnd'] = cor_data[0][:, qubit+127]
        all_memories_dbl[qubit]['mmr_1_scnd'] = cor_data[1][:, qubit+127]

        all_memories_dbl_X[qubit]['mmr_0'] = cor_data[2][:, qubit]
        all_memories_dbl_X[qubit]['mmr_1'] = cor_data[3][:, qubit]
        all_memories_dbl_X[qubit]['mmr_0_scnd'] = cor_data[2][:, qubit+127]
        all_memories_dbl_X[qubit]['mmr_1_scnd'] = cor_data[3][:, qubit+127]

    # Post select the calibration jobs!
    proc_all_memories_dbl, gmm_dict, msmt_err_dict = postselect_calib_data(all_memories_dbl, x_between_meas=False)
    proc_all_memories_dbl_X, gmm_dict_X, msmt_err_dict_X = postselect_calib_data(all_memories_dbl_X, x_between_meas=True)

    # Get the KDEs
    bandwidths = [0.6]
    num_points = 51

    kde_dict_X = csi.get_KDEs(proc_all_memories_dbl_X, bandwidths, relError=1, absError=-1, num_points=num_points) # using data without leakage!

    # Qubit lists
    code_qubits = set()
    ancilla_qubits = set()
    for link in params.links:
        code_qubits.add(link[0])
        code_qubits.add(link[2])
        ancilla_qubits.add(link[1])
    code_qubits = list(code_qubits)
    ancilla_qubits = list(ancilla_qubits)

    # Get qubit mapping
    memory = cor_data[4]
    qubit_mapping = {}
    for idx in range(memory.shape[1]):
        if idx < params.T * len(ancilla_qubits):
            qubit_mapping[idx] = ancilla_qubits[idx % len(ancilla_qubits)]
        else:
            qubit_mapping[idx] = code_qubits[idx - params.T * len(ancilla_qubits)]
    inverted_q_map = inv_qubit_mapping(qubit_mapping)


    # Get the matrices
    threshold = 0.05
    pSoft, countMat, estim0Mat, estim1Mat = csi.iqConvertorEstim(memory, inverted_q_map, kde_dict_X, relError=1, absError=-1)
    weightMat = -np.log(pSoft/(1-pSoft))    
    pSoft_trunc, ratio = process_pSoft(pSoft, estim0Mat, estim1Mat, threshold)

    # Save the data
    mat_dict = {
        'pSoft': pSoft, # the probability of a soft flip (ndarray)
        'countMat': countMat, # the estimated outcome of the IQ point (ndarray)
        'estim0Mat': estim0Mat, # the probability to sample the IQ point if the qubit is in state 0 (ndarray)
        'estim1Mat': estim1Mat, # the probability to sample the IQ point if the qubit is in state 1 (ndarray)
        'weightMat': weightMat, # the weight -np.log(pSoft/(1-pSoft)) (ndarray)
        'pSoft_trunc': pSoft_trunc, # the pSoft matrix with "leakage" points as maximally ambiguous (ndarray)
        'leakage proportion': ratio, # the proportion of classified IQ points (float)
        'msmt_err_dict': msmt_err_dict_X, # the measurement error dictionary (dict) {qubit: {ph_mean:, ps_mean:, ph0:, ph1:, ps0:, ps1:}}
    }
    with open('data/matrices_X/' + file, 'wb') as f:
        pickle.dump(mat_dict, f)

100%|██████████| 4/4 [02:34<00:00, 38.74s/it]


In [2]:
for key, value in mat_dict.items():
    print(key, value.shape)

pSoft (1000, 7154)
countMat (1000, 7154)
estim0Mat (1000, 7154)
estim1Mat (1000, 7154)
weightMat (1000, 7154)
pSoft_trunc (1000, 7154)
leakage proportion ()


AttributeError: 'dict' object has no attribute 'shape'